<a href="https://colab.research.google.com/github/veruizr/micropython/blob/main/4bar_gendataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Este programa genera un dataset para entrenamiento de redes neuronales para predicción de posición de mecanismos de 4 barras. Utiliza libreria math para el cálculo de funciones trigonométricas la cual es funcional en micropython. Teniendo en cuenta la capacidad de los microcontroladores se genera por corrida un archivo .csv de 200 muestras que puede transmitirse al equipo por memoria flash/usb

In [ ]:
import math

# Se crea una clase para generar números aleatorios que permitan generar el
# conjunto de entradas de longitudes y ángulo. Evita usar la librería random
class FloatLCG:
    def __init__(self, seed=12345):
        self.state = seed
        self.a = 1103515245
        self.c = 12345
        self.m = 2**31  # 2147483648

    def random(self):
        # Genera un número flotante entre 0 y 1
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m

    def uniform(self, low, high):
        # Genera un número flotante entre `low` y `high`
        return low + (high - low) * self.random()


##Programa micropython para calcular la posición de mecanismos de cuatro barras
# usando el método de Newton- Raphson

#cálculo de condición de Grashoff y tipo de mecanismo de acuerdo con clasificación de Barker
def id_grashof(L1, L2, L3, L4):

    longitudes = sorted([L1, L2, L3, L4])#organizar longitudes de menor a mayor
    s,p,q,l = longitudes #s=eslabón más corto,l=eslabón más largo, p y q longitudes intermedias
    suma_extremos = s + l
    suma_intermedias = p + q
    gr=True

    if suma_extremos < suma_intermedias:
        gr=True
        # Determinar qué eslabón es el más corto
        eslabones = [(L1, "fijo"), (L2, "entrada"),
                        (L3, "acoplador"), (L4, "salida")]
        min_eslabon = min(eslabones, key=lambda x: x[0])

        if min_eslabon[1] == "entrada":
                return "Grashof, manivela-balancín"
        elif min_eslabon[1] == "fijo":
                return "Grashof, doble manivela"
        elif min_eslabon[1] == "acoplador":
                return "Grashof, doble balancín"
        else:
                return "manivela-balancín"

    elif suma_extremos == suma_intermedias:
          gr=False
          return "NO Grashof, mecanismo plegable"
    else:
          return "NO Grashof, triple balancín"


#Función para calcular posición mediante Newton Raphson

def four_bar_analysis(L1, L2, L3, L4, theta2, max_iter=100, tol=1e-6):
    """
    Calcula los ángulos de los eslabones acoplador y de salida.
    Retorna: (theta3, theta4, near_singular, grashof)
    """
    # Verificar condición de Grashof
    grashof = id_grashof(L1, L2, L3, L4)

    # Estimación inicial para theta4 usando una ecuación de distancia entre
    #el apoyo fijo izquierdo y el punto de unión acoplador-salida

    theta4 = theta2  # Valor inicial
    for _ in range(50):
        dx = L1 - L4*math.cos(theta4) - L2*math.cos(theta2)
        dy = L4*math.sin(theta4) - L2*math.sin(theta2)
        error = dx**2 + dy**2 - L3**2
        if abs(error) < 1e-3:
            break
        df = 2*dx*(-L4*math.sin(theta4)) + 2*dy*(L4*math.cos(theta4))
        if df == 0:
            break
        theta4 -= error/df

    # Estimación inicial para theta3
    sin_theta3 = (L4*math.sin(theta4) - L2*math.sin(theta2))/L3
    cos_theta3 = (L1 - L4*math.cos(theta4) - L2*math.cos(theta2))/L3
    theta3 = math.atan2(sin_theta3, cos_theta3)

    # Método de Newton-Raphson para solución del sistema de ecuaciones de posición con detección de singularidades
    near_singular = False
    for _ in range(max_iter):
        # # Ecuación de lazo: r2 + r3 = r1 + r4 se asume theta1=0
        # Componente X: L2*cos(θ2) + L3*cos(θ3) = L1*cos(0) + L4*cos(θ4)
        # Componente Y: L2*sin(θ2) + L3*sin(θ3) = L1*sin(0) + L4*sin(θ4)
        lx = L2*math.cos(theta2) + L3*math.cos(theta3) - L4*math.cos(theta4) - L1
        ly = L2*math.sin(theta2) + L3*math.sin(theta3) - L4*math.sin(theta4)

        if abs(lx) < tol and abs(ly) < tol:
            break

        # Calcular Jacobiano del sistema de ecuaciones
        # Derivadas parciales de f respecto a theta3 y theta4
        J11 = -L3*math.sin(theta3)
        J12 = L4*math.sin(theta4)
        # Derivadas parciales de g respecto a theta3 y theta4
        J21 = L3*math.cos(theta3)
        J22 = -L4*math.cos(theta4)
        #determinante del Jacobiano
        det = J11*J22 - J12*J21

        # Verificar singularidad
        if abs(det) < 1e-3:
            near_singular = True

        if det == 0:
            return None, None, True, grashof # Matriz singular

        # Calcular incrementos
        delta3 = (-lx*J22 + ly*J12)/det
        delta4 = (lx*J21 - ly*J11)/det

        theta3 += delta3
        theta4 += delta4

        conf=0
        if theta3 is not None and theta3 < 0:
          configuracion='Cruzada'
          conf=1
        else:
          configuracion='Abierta'
        conf=0
    else:
        return None, None, near_singular, grashof

    # Verificación final de singularidad
    J11 = -L3*math.sin(theta3)
    J12 = L4*math.sin(theta4)
    J22 = -L4*math.cos(theta4)
    det = J11*J22 - (L4*math.sin(theta4))*(L3*math.cos(theta3))
    if abs(det) < 1e-3:
        near_singular = True

    return theta3, theta4, near_singular, grashof

# Generar dataset compatible en micropython

def generate_micro_dataset(num_samples, filename):

    # Crear archivo CSV
    with open(filename, 'w') as f:
        # Escribir encabezados
        f.write('L1,L2,L3,L4,theta2,theta3,theta4,grashof,near_singular\n,configuracion\n')

        samples_generated = 0
        rng = FloatLCG(seed=2025)
        while samples_generated < num_samples:
            # Generar longitudes aleatorias (0.5-10.0 unidades)

            L1 = rng.uniform(0.5, 10.0)
            L2 = rng.uniform(0.5, 10.0)
            L3 = rng.uniform(0.5, 10.0)
            L4 = rng.uniform(0.5, 10.0)

            # Verificar si se puede formar un mecanismo (desigualdad del triángulo)
            if (L1 + L2 + L3 < L4) or (L1 + L2 + L4 < L3) or \
               (L1 + L3 + L4 < L2) or (L2 + L3 + L4 < L1):
                continue

            # Generar ángulo de entrada aleatorio
            theta2 = rng.uniform(0, 2*math.pi)

            # Calcular posición del mecanismo
            theta3, theta4, near_singular,conf = four_bar_analysis(L1, L2, L3, L4, theta2)

            # Si no hay solución válida, saltar muestra
            if theta3 is None:
                continue

            # Determinar condición de Grashof
            gr = id_grashof(L1, L2, L3, L4)
            grashof = 1 if gr == True else 0

            # Escribir fila en CSV
            f.write(f'{L1:.4f},{L2:.4f},{L3:.4f},{L4:.4f},')
            f.write(f'{theta2:.4f},{theta3:.4f},{theta4:.4f},')
            f.write(f'{grashof},{1 if near_singular else 0}\n')

            samples_generated += 1
            if samples_generated % 10 == 0:
                print(f'Muestras: {samples_generated}/{num_samples}')

    return samples_generated

# Configuración de dataset para microcontrolador con MicroPython
NUM_SAMPLES = 200  # Tamaño manejable para microcontroladores
FILE_NAME = 'fourbar_data.csv'

# Generar dataset
print("\nGenerando dataset para mecanismo de 4 barras...")
samples_created = generate_micro_dataset(NUM_SAMPLES, FILE_NAME)
print(f"\nDataset creado: {FILE_NAME}")
print(f"Muestras generadas: {samples_created}")

# Verificar tamaño del archivo
try:
    with open(FILE_NAME, 'r') as f:
        content = f.read()
        file_size = len(content)
        print(f"Tamaño del archivo: {file_size} bytes")
        print(f"Tamaño promedio por muestra: {file_size/samples_created:.1f} bytes")
except Exception as e:
    print(f"Error al verificar tamaño: {e}")



Generando dataset para mecanismo de 4 barras...
Muestras: 10/200
Muestras: 20/200
Muestras: 30/200
Muestras: 40/200
Muestras: 50/200
Muestras: 60/200
Muestras: 70/200
Muestras: 80/200
Muestras: 90/200
Muestras: 100/200
Muestras: 110/200
Muestras: 120/200
Muestras: 130/200
Muestras: 140/200
Muestras: 150/200
Muestras: 160/200
Muestras: 170/200
Muestras: 180/200
Muestras: 190/200
Muestras: 200/200

Dataset creado: fourbar_data.csv
Muestras generadas: 200
Tamaño del archivo: 11057 bytes
Tamaño promedio por muestra: 55.3 bytes
